## LangChain Expression Language (LCEL) Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

1. What is LCEL?

LCEL stands for LangChain Expression Language.
It provides a concise, composable, and Pythonic way to build chains, pipelines, and workflows in LangChain.

Goal: Replace the older SequentialChain, RouterChain, etc., with a more flexible and declarative API.
Key Concepts: Chains as functions (or callables), easy composition (| operator), clear input/output typing.

2. Core LCEL Components

a. Runnable
The core abstraction in LCEL.
Any object that implements the .invoke() method and can be composed using the | operator.
Examples: Models, PromptTemplates, Chains.

b. PromptTemplate
Used to format inputs into prompts for LLMs or ChatModels.
LCEL expects prompt templates to have variable names matching the input keys.

c. LLM and ChatModel
Language Models and ChatModels from langchain_openai, langchain_community, etc.
Compatible with LCEL if they implement .invoke().

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1183d38d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11839d250>, root_client=<openai.OpenAI object at 0x1185601d0>, root_async_client=<openai.AsyncOpenAI object at 0x11925c550>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [3]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)

question = template.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})

response = llm.invoke(question)
print(response.content)    

1. The solar system consists of the sun, eight planets, and various other celestial bodies.
2. The eight planets in order from the sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
3. The sun is a star at the center of the solar system, providing light and heat to the planets.
4. The planets orbit the sun in elliptical paths, with each planet having its own unique characteristics.
5. The solar system also includes moons, asteroids, comets, and dwarf planets like Pluto.


In [4]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)

chain = template | llm


In [5]:
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response.content)

1. The solar system consists of the sun, eight planets, and various other celestial bodies.
2. The eight planets in order from the sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
3. The sun is a star at the center of the solar system, providing light and heat to the planets.
4. The planets orbit the sun in elliptical paths, with each planet having its own unique characteristics.
5. The solar system also includes moons, asteroids, comets, and dwarf planets like Pluto.


In [6]:
response = chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 5})
print(response.content)

1. The solar system consists of the Sun, eight planets, moons, asteroids, comets, and other celestial objects.
2. The Sun is at the center of the solar system, with planets orbiting around it.
3. The four inner planets are Mercury, Venus, Earth, and Mars, while the four outer planets are Jupiter, Saturn, Uranus, and Neptune.
4. The asteroid belt lies between Mars and Jupiter, and comets are icy bodies that orbit the Sun.
5. The solar system formed about 4.6 billion years ago from a giant cloud of gas and dust.


# What is StrOutputParser?


StrOutputParser is a utility class in LangChain for handling the output of language models (LLMs/ChatModels).
It parses the model’s response and returns it as a plain Python string, removing any unnecessary metadata, wrappers, or objects that the model or API may return.


## Why do we need it?

- By default, LLMs or chat models in LangChain might return structured objects, message objects, or dictionaries.
- In most cases, especially for simple chains, you just want the generated text string (e.g., the answer, summary, completion, etc.).
- StrOutputParser extracts this text from the raw output so your chain returns a clean string.


In [29]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("What is the capital of {country}?")
llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

chain1 = prompt | llm 
chain2 = prompt | llm | parser

print("Without StrParser\n")
resultChain1 = chain1.invoke({"country" , "India"})
print(resultChain1)

print("\nWith StrParser\n")
resultChain2 = chain2.invoke({"country": "India"})
print(resultChain2)  # Output: "The capital of India is New Delhi."


Without StrParser

content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 19, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BvIPmqyz8mGlRObRn0tI2tYEE6Pz4', 'finish_reason': 'stop', 'logprobs': None} id='run-ecade9f1-3bf5-4621-8c53-5dfaee04de18-0' usage_metadata={'input_tokens': 19, 'output_tokens': 8, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

With StrParser

The capital of India is New Delhi.


In [7]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response)

1. The solar system consists of the sun, eight planets, and various other celestial bodies.
2. The eight planets in order from the sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
3. The sun is a star at the center of the solar system, providing light and heat to the planets.
4. The planets orbit the sun in elliptical paths, with each planet having its own unique characteristics.
5. The solar system also includes moons, asteroids, comets, and dwarf planets like Pluto.


### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?


# Concept: What Are Runnables and Chaining?

Runnables in LangChain are modular, composable components—like prompts, LLMs, output parsers, or even custom functions—that implement an .invoke() method.

Chaining Runnables means connecting these modular components in sequence, so the output of one becomes the input of the next. In LCEL, you use the pipe operator (|) to compose them functionally, much like Unix pipes or Pandas method chaining.



In [9]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='tell me about the {topics} in {points} points'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1183d38d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11839d250>, root_client=<openai.OpenAI object at 0x1185601d0>, root_async_client=<openai.AsyncOpenAI object at 0x11925c550>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [10]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                   ''')

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response': response})
print(output)

The text is relatively easy to understand as it provides basic information about the solar system and its components.


In [12]:
composed_chain = {"response": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 5})
print(output)

The text is relatively easy to understand as it provides basic information about the solar system and its components.


### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.


### What is a Parallel Chain?

Parallel chaining lets you run multiple chains (pipelines) simultaneously using the same or similar inputs, collecting all outputs together.

In LangChain LCEL, this is achieved via RunnableParallel.

It’s analogous to a “fan-out” pattern, where you branch out your input to multiple tasks and gather all their results in one go.

### Why Use Parallel Chains?

- To answer multiple questions about the same input in one shot.
- To generate diverse content (e.g., facts, poems, summaries) from the same base information.
- To save code and avoid running chains sequentially when there’s no dependency between them.




In [13]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)
fact_chain = template | llm | StrOutputParser()

output = fact_chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2})
print(output)

1. The solar system consists of the sun, eight planets, moons, asteroids, comets, and other celestial objects.
2. The planets in the solar system orbit around the sun in elliptical paths.


In [14]:
question = HumanMessagePromptTemplate.from_template('write a poem on {topics} in {sentences} lines')


messages = [system, question]
template = ChatPromptTemplate(messages)
poem_chain = template | llm | StrOutputParser()

output = poem_chain.invoke({'school': 'primary', 'topics': 'solar system', 'sentences': 2})
print(output)

Planets dance in space,
Sun's light gives them grace.


What happens here?

- The same input dictionary is sent to both fact_chain and poem_chain.
- Both chains execute independently (in parallel in logic, not necessarily true multi-threaded parallelism).
- Output is a dictionary:

{'fact': <fact_output>, 'poem': <poem_output>}

In [15]:
from langchain_core.runnables import RunnableParallel
chain = RunnableParallel(fact = fact_chain, poem = poem_chain)


output = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2})
print(output['fact'])
print('\n\n')
print(output['poem'])

1. The solar system consists of the sun, eight planets, moons, asteroids, comets, and other celestial objects.
2. The planets in the solar system orbit around the sun in elliptical paths.



Planets dance in space,
Sun's light gives them grace.


### Chain Router
- The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [16]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Review: {review}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
# review = "I am not happy with the service. It is not good."
chain.invoke({'review': review})

'Positive'

In [17]:
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [18]:
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Review: {review}
                Answer:"""


negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [19]:
def rout(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [20]:
from langchain_core.runnables import RunnableLambda

full_chain = {"sentiment": chain, 'review': lambda x: x['review']} | RunnableLambda(rout)

full_chain


{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='Given the user review below, classify it as either being about `Positive` or `Negative`.\n            Do not respond with more than one word.\n\n            Review: {review}\n            Classification:'), additional_kwargs={})])
             | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1183d38d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11839d250>, root_client=<openai.OpenAI object at 0x1185601d0>, root_async_client=<openai.AsyncOpenAI object at 0x11925c550>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(rout)

In [ ]:
# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

output = full_chain.invoke({'review': review})
print(output)

Dear valued customer,

We are truly sorry to hear that you are not happy with the service provided. Your satisfaction is our top priority and we apologize for any inconvenience caused.

We would like to hear more about your experience and address your concerns in a timely manner. Please feel free to share your feedback with us via email at udemy@kgptalkie.com. Your input is highly valuable to us and we are committed to improving our services based on your feedback.

Thank you for bringing this to our attention and giving us the opportunity to make things right.

Best regards,
[Your Name]


### Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
- This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.



In [22]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explain these inputs in 5 sentences: {input1} and {input2}")

prompt


ChatPromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, template='Explain these inputs in 5 sentences: {input1} and {input2}'), additional_kwargs={})])

In [23]:
chain = prompt | llm | StrOutputParser()

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

1. The input "Earth is planet" is stating that Earth is a celestial body that orbits around the Sun, similar to other planets in our solar system.
2. The input "Sun is star" is indicating that the Sun is a massive, luminous sphere of plasma that emits light and heat, just like other stars in the universe.
3. Both inputs are basic astronomical facts that are commonly taught in science classes and are fundamental to understanding the structure and dynamics of our solar system.
4. The relationship between Earth and the Sun is crucial for sustaining life on our planet, as the Sun provides the energy necessary for photosynthesis and other essential processes.
5. Understanding the roles of Earth as a planet and the Sun as a star helps us appreciate the vastness and complexity of the universe and our place within it.


In [24]:
chain = prompt | llm | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 
                                            'word_counts': RunnableLambda(word_counts), 
                                            'output': RunnablePassthrough()}

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

{'char_counts': 701, 'word_counts': 122, 'output': '1. The input "Earth is planet" is stating that Earth is a celestial body that orbits around the Sun, similar to other planets in our solar system.\n2. The input "Sun is star" is indicating that the Sun is a massive, luminous sphere of plasma held together by its own gravity, much like the billions of other stars in the universe.\n3. Both inputs are basic astronomical facts that are widely accepted by scientists and astronomers.\n4. The relationship between Earth and the Sun is that Earth revolves around the Sun, which provides the heat and light necessary for life on our planet.\n5. Understanding these inputs helps us comprehend the vastness and complexity of the universe and our place within it.'}


### Custom Chain using `@chain` decorator

Custom Chains with @chain Decorator in LangChain (LCEL, v0.3.x)

1. What is the `@chain` Decorator?

- The @chain decorator is a feature of LangChain’s LCEL API (Expression Language).
- It transforms any standard Python function into a Runnable Chain.
- This allows you to wrap arbitrary Python logic—including loops, conditionals, and composition of other chains—into a component that behaves just like any other chain in LCEL (i.e., it supports .invoke(), .batch(), etc.).

2. Why Use `@chain`?

For complex, custom workflows that can’t be easily expressed by chaining with the | operator.
To combine multiple sub-chains, add custom logic, or perform steps that need Python code (like aggregation, formatting, conditional logic).
It enables full flexibility within the LCEL framework: your custom function becomes a first-class Runnable.

3. Example Explained


4. How Does it Work?

The decorator wraps your function into a Runnable-compatible object.
Supports all LCEL operations (invoke, batch, etc.).
You can include arbitrary Python code—loops, conditionals, error handling, etc.
The function input is a single parameter (often a dict).
The output can be any object—dict, list, string, etc.




In [25]:
from langchain_core.runnables import chain

@chain
def custom_chain(params):
    return {
        'fact': fact_chain.invoke(params),
        'poem': poem_chain.invoke(params),
    }




4. How Does it Work?

- The decorator wraps your function into a Runnable-compatible object.
- Supports all LCEL operations (invoke, batch, etc.).
- You can include arbitrary Python code—loops, conditionals, error handling, etc.
- The function input is a single parameter (often a dict).
- The output can be any object—dict, list, string, etc.


In [26]:

params = {'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2}
output = custom_chain.invoke(params)
print(output['fact'])
print('\n\n')
print(output['poem'])

1. The solar system consists of the sun, eight planets, moons, asteroids, comets, and other celestial objects.
2. The planets in the solar system orbit around the sun in elliptical paths.



Planets dance in space,
Sun's light gives them grace.
